# To Dos
* ESPN Website not showing - if persists use the worldfootball.com website instead

* Using the English Version of the website the new column in the DF can be created by using following statement: dt_format = datetime.datetime.strptime(date + ' ' + time, '%a, %b %d %I:%M %p'); Needs to set year to 2020

* [Last Step] Implement in PytonEverywhere

# Tests - Web Scrap

In [47]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [48]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

page = "https://www.espn.com/soccer/team/fixtures/_/id/819/flamengo"
# alternative website
#page = "https://www.worldfootball.net/teams/flamengo-rj/2020/3/"
pageTree = requests.get(page, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

In [49]:
#access table-header of pageSoup
full_table =  pageSoup.find_all("table") 

In [54]:
full_table

[]

In [50]:
# copy all rows into a list 
round_list = []
row =  pageSoup.find_all("tr")
count = 0
for row in full_table:
    for td in row.find_all("td"): 
        # if empty row or 'v' found we ignore it
        if (td.text != '') and (td.text != 'v'):
            round_list.append(td.text)               

In [51]:
import numpy as np

# rehape the list in array of 5 elements per round
n_rounds = int(len(round_list)/5)
round_list_rehape = np.reshape(round_list, (n_rounds, 5))

In [52]:
df_rounds = pd.DataFrame(data=round_list_rehape,columns=['date', 'home', 'away', 'time', 'competition'])

In [42]:
# Upper case 'Flamengo' --> 'FLAMENGO' :-)
df_rounds['home'] = df_rounds['home'].str.replace('Flamengo' ,'FLAMENGO')
df_rounds['away'] = df_rounds['away'].str.replace('Flamengo' ,'FLAMENGO')

In [43]:
#gets both columns 'date' & 'time' and returnes formated dt object
def format_time(date, time):
    import locale
    import datetime
    # converts date & time to one datename object
    dt_format = datetime.datetime.strptime(date + ' ' + time, '%a, %b %d %I:%M %p')
    #dt_format = dt_format.replace(year=2020)
    #dt_format = dt_format + timedelta(hours=12)
    
    return dt_format

In [44]:
# create new column for combined date_time
df_rounds['date_time'] = df_rounds.apply(lambda row: format_time(row['date'], row['time']), axis=1)

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [ ]:
df_rounds

# Tests - iCal & Flask

In [ ]:
from icalendar import Calendar, Event
import datetime

In [ ]:
cal = Calendar()

cal.add('prodid', '-//My calendar product//mxm.dk//')
cal.add('version', '2.0')

In [ ]:
for i in df_rounds.index:
    event = Event()
    event.add('summary', df_rounds['home'][i] + ' x ' + df_rounds['away'][i])
    event.add('location', df_rounds['competition'][i])
    # +4 for CET time + 12 PM
    event.add('dtstart', df_rounds['date_time'][i] + datetime.timedelta(hours=16))
    # +6 = +4 (CET time) +2 (game lentgh) + 12 PM
    event.add('dtend', df_rounds['date_time'][i] + datetime.timedelta(hours=18))
    # write evento to calendar element
    cal.add_component(event)


In [ ]:
#write to ical file
#f = open('ovo.ics', 'wb')
#f.write(cal.to_ical())
#f.close()

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def index():
    return cal.to_ical()

if __name__ == '__main__':
    app.run()

In [55]:
#TESTING

In [12]:
import datetime
date = 
dt_format = datetime.datetime.strptime(date + ' ' + time, '%a, %b %d %I:%M %p')

In [13]:
dt_format

datetime.datetime(1900, 3, 11, 20, 30)